<a href="https://colab.research.google.com/github/ovapichi/biomecanica-datos/blob/main/McGill_y_Celda_sinforniif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fpdf


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=8aa4480b9f73e9aff06bc3caf900d244ad7f1f89ab890e619fde1441da732ba1
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [ ]:
#setear todo lo necesario
import fpdf as FPDF
from datetime import datetime
import pandas as pd
import numpy as np
from fpdf import FPDF
from google.colab import drive, files
import shutil
drive.mount('/content/drive')
direccion='/content/drive/Shareddrives/Ova/Vertiz/Saltos sep 2023/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#encabezado y pie de página del pdf
class PDF (FPDF):
  def header(self):
    self.image("/content/drive/Shareddrives/Ova/Vertiz/Movement Science.png",2,0.5,2.5) #Cambiar Logos
    self.image("/content/drive/Shareddrives/Sociedad Hebraica Argentina/Imágenes/logo.png",15,0.5,2.5)
    self.set_font("arial","B",12)
    self.set_text_color(50,175,175)
    self.set_font("arial","B",10)
    self.ln(12)
  def footer(self):
    self.set_y(-1)
    self.set_font("arial","",12)
    self.set_text_color(0,0,0)
    self.cell(0,1,f'{self.page_no()}',align="R")
  def set_col(self, col):
  # Set column position:
    self.col = col
    x = 0.50 + col * 2.5
    self.set_left_margin(x)
    self.set_x(x)
#Carátula del pdf
def caratula(nombre,protocolo):
  now = datetime.now()
  hoy=now.date()
  profesionales= ["Lic. Toscano, Ana Paula",'Lic. García Piccinini, Osvaldo','Med. Hidalgo, Esteban']
  pdf.set_y(4)
  #pdf.set_text_color(50,175,175)
  pdf.set_font("arial","",12)
  pdf.image("/content/drive/Shareddrives/Sociedad Hebraica Argentina/Imágenes/informe.png",12,5,10)
  pdf.cell(0,5,f'Dirigido a: {nombre}' ,ln=2, align='l')
  pdf.multi_cell(h=2, align='l', w=10, txt=f'Protocolo: {protocolo}\n')
  pdf.cell(0,5,f'{hoy}',ln=True, align='l')
  pdf.add_page()
  pdf.set_y(4)
  pdf.set_text_color(0,0,0)
  pdf.cell(2,2,f'Equipo profesional',ln=True, align='l')
  for crack in profesionales:
    pdf.cell(2,2,f'{crack}',ln=True, align='l')
  pdf.image("/content/drive/Shareddrives/Sociedad Hebraica Argentina/Imágenes/firmas lab.jpg",5.5,16,10)
  pdf.set_font("arial","",10)
  pdf.multi_cell(h=0.5, align='L', w=0, txt='El equipo responsable del Laboratorio de Biomecánica Análisis Funcional del Movimiento y Rendimiento Humano queda a su disposición para el análisis conjunto del caso, su interpretación, tanto como las eventuales consideraciones de las conclusiones. \nAgradecemos la confianza al permitirnos realizar esta evaluación. \nLo saludamos cordialmente.')
  return
  # =================================BLOQUE DE FUNCIONES=================================================



In [ ]:
#Leer csv con pandas
#df=pd.read_csv('/content/drive/Shareddrives/Ova/sha grupal/Planilla de SHA marzo - data.csv')
df=pd.read_csv(direccion +'datosBosco.csv')
df.rename(columns = {'Unnamed: 0':'celda'}, inplace = True)#para convertir a dataframe
data = pd.DataFrame(df)
print(data)
data=data.sort_values('nombres')
#data=data.set_index('categoria')
data=data.sort_index()
#print(data)
#cambio el tipo de dato a float
data['celda']= data['celda'].astype(bool)
data['isqIzq'] = data['isqIzq'].astype(float)*data['celda']
data['isqDer'] = data['isqDer'].astype(float)*data['celda']
data['cuadIzq'] = data['cuadIzq'].astype(float)*data['celda']
data['cuadDer'] = data['cuadDer'].astype(float)*data['celda']
#visualizo el tipo de dato de cada columna
#print(data.dtypes)


In [ ]:
pdf = PDF("p","cm","A4")
pdf.add_page()
pdf.set_font("Arial",size=10)

caratula("Club Vertiz Voley", "Evaluación de la fuerza resistencia por protocolo McGill y evaluación de la fuerza isométrica de MMII con celda de cargas")
#ctrl+D (selecciona toda la palabra)  ctrl+shift+L  (Seleccionar la misma selección para cambiarla en todo el código)

#calculo del indice de simetria
data['simetriaLateral']=((data['derecho']/data['izquierdo'])*100-100)
#tomo solo el 1er decimal del valor
data['simetriaLateral']=data['simetriaLateral'].round(1)
#selecciono los valores optimos al 5% leves <50%
simetricoLat = (data['simetriaLateral'] <=5) & (data['simetriaLateral'] >=-5)
izquierdoLat = (data['simetriaLateral'] <=-5) & (data['simetriaLateral']>=-50)
derechoLat = (data['simetriaLateral'] >=5) & (data['simetriaLateral']<=50)

pdf.add_page()
pdf.set_font("Arial",size=10, style="B")
pdf.set_col(2.5)
pdf.set_y(2)
pdf.multi_cell(h=0.35, align='L', w=0, txt=f"1) ZONA MEDIA\n 1.1) SIMETRIA LATERAL")

pdf.set_col(0)
pdf.set_y(2.5)
pdf.cell(2,2,f'Valores simétricos \n',ln=False, align='l')
pdf.set_col(2.5)
pdf.cell(2,2,f'Fortalecer lado derecho\n',ln=False, align='l')
pdf.set_col(5)
pdf.cell(2,2,f'Fortalecer lado izquierdo \n',ln=True, align='l')

pdf.set_y(4)
pdf.set_font("Arial",size=8)
pdf.set_col(0)
if len(data['nombre'][simetricoLat])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f" {data['nombre'][simetricoLat]}")
pdf.set_y(4)
pdf.set_col(2.5)
if len(data['nombre'][izquierdoLat])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +25%  \n {data['nombre'][izquierdoLat]} \n ")
if len(data['nombre'][data['simetriaLateral']<-50])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Acentuada +50%\n  {data['nombre'][data['simetriaLateral']<-50]}\n")
pdf.set_y(4)
pdf.set_col(5)
if len(data['nombre'][derechoLat])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +25%  {data['nombre'][derechoLat]} \n  ")
if len(data['nombre'][data['simetriaLateral'] >50])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Acentuada +50%\n  {data['nombre'][data['simetriaLateral'] >50]}\n")

data['simetriaAp']=(data['anterior']/data['posterior'])*100-72*data['ramaF']-84*data['ramaM']
data['simetriaAp']=data['simetriaAp'].round(1)
simetricoAp=(data['simetriaAp'] <=10) & (data['simetriaAp'] >=-10)
asimetricoAnt=(data['simetriaAp'] <-10) & (data['simetriaAp'] >=-50)
asimetricoPost=(data['simetriaAp'] > 10) & (data['simetriaAp'] <50)

pdf.add_page()
pdf.set_font("Arial",size=10, style="B")
pdf.set_col(2.5)
pdf.set_y(2)
pdf.multi_cell(h=0.35, align='L', w=0, txt=f"1.2) SIMETRIA ANTEROPOSTERIOR")

pdf.set_col(0)
pdf.set_y(2.5)
pdf.cell(2,2,f'Valores simétricos \n',ln=False, align='l')
pdf.set_col(2.5)
pdf.cell(2,2,f'Fortalecer lado anterior (Adelante)\n',ln=False, align='l')
pdf.set_col(5)
pdf.cell(2,2,f'Fortalecer lado posterior (Atrás) \n',ln=True, align='l')

pdf.set_y(4)
pdf.set_font("Arial",size=8)
pdf.set_col(0)
if len(data['nombre'][simetricoAp])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f" {data['nombre'][simetricoAp]}")
pdf.set_y(4)
pdf.set_col(2.5)
if len(data['nombre'][asimetricoAnt])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +25%  \n {data['nombre'][asimetricoAnt]} \n  ")
if len(data['nombre'][data['simetriaAp']<-50])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Acentuada +50%\n  {data['nombre'][data['simetriaAp']<-50]}\n")

pdf.set_y(4)
pdf.set_col(5)
if len(data['nombre'][asimetricoPost])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +25%  {data['nombre'][asimetricoPost]}  \n")
if len(data['nombre'][data['simetriaAp'] >50])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Acentuada +50%\n  {data['nombre'][data['simetriaAp'] >50]}\n")

data['simetriaDe']=(data['derecho']/data['posterior'])*100-40*data['ramaF']-58*data['ramaM']
data['simetriaDe']=data['simetriaDe'].round(1)
simetricoDe=(data['simetriaDe'] <=10) & (data['simetriaDe'] >=-10)
asimetricoDer=(data['simetriaDe'] <-10) & (data['simetriaDe'] >-50)
asimetricoDPost=(data['simetriaDe'] >10) & (data['simetriaDe'] <50)

pdf.add_page()
pdf.set_font("Arial",size=10, style="B")
pdf.set_col(2.5)
pdf.set_y(2)
pdf.multi_cell(h=0.35, align='L', w=0, txt=f"1.3) SIMETRIA POSTERODERECHA ")

pdf.set_col(0)
pdf.set_y(2.5)
pdf.cell(2,2,f'Valores simétricos \n',ln=False, align='l')
pdf.set_col(2.5)
pdf.cell(2,2,f'Fortalecer lado derecho \n',ln=False, align='l')
pdf.set_col(5)
pdf.cell(2,2,f'Fortalecer lado posterior (Atrás) \n',ln=True, align='l')

pdf.set_y(4)
pdf.set_font("Arial",size=8)
pdf.set_col(0)
if len(data['nombre'][simetricoDe])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f" {data['nombre'][simetricoDe]}")
pdf.set_y(4)
pdf.set_col(2.5)
if len(data['nombre'][asimetricoDer])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +25%  \n {data['nombre'][asimetricoDer]} \n  ")
if len(data['nombre'][data['simetriaDe']<-50])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Acentuada +50%\n  {data['nombre'][data['simetriaDe']<-50]}\n")
pdf.set_y(4)
pdf.set_col(5)

if len(data['nombre'][asimetricoDPost])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +25%  {data['nombre'][asimetricoDPost]} \n  ")
if len(data['nombre'][data['simetriaDe'] >50])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Acentuada +50%\n  {data['nombre'][data['simetriaDe'] >50]}\n")

data['simetriaIz']=(data['izquierdo']/data['posterior'])*100-42*data['ramaF']-61*data['ramaM']
data['simetriaIz']=data['simetriaIz'].round(1)
simetricoIz=(data['simetriaIz'] <=10) & (data['simetriaIz'] >=-10)
asimetricoIzq=(data['simetriaIz'] <-10) & (data['simetriaIz'] >-50)
asimetricoIPost=(data['simetriaIz'] >10) & (data['simetriaIz'] <50)

pdf.add_page()
pdf.set_font("Arial",size=10, style="B")
pdf.set_col(2.5)
pdf.set_y(2)
pdf.multi_cell(h=0.35, align='L', w=0, txt=f"1.4) SIMETRIA POSTEROIZQUIERDA\n  ")

pdf.set_col(0)
pdf.set_y(2.5)
pdf.cell(2,2,f'Valores simétricos \n',ln=False, align='l')
pdf.set_col(2.5)
pdf.cell(2,2,f'Fortalecer lado izquierdo \n',ln=False, align='l')
pdf.set_col(5)
pdf.cell(2,2,f'Fortalecer lado posterior (Atrás) \n',ln=True, align='l')


pdf.set_y(4)
pdf.set_font("Arial",size=8)
pdf.set_col(0)
if len(data['nombre'][simetricoIz])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f" {data['nombre'][simetricoIz]}")
pdf.set_y(4)
pdf.set_col(2.5)
if len(data['nombre'][asimetricoIzq])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +25%  \n {data['nombre'][asimetricoIzq]} \n  ")
if len(data['nombre'][data['simetriaIz']<-50])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Acentuada +50%\n  {data['nombre'][data['simetriaIz']<-50]}\n")
pdf.set_y(4)
pdf.set_col(5)

if len(data['nombre'][asimetricoIPost])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +25%  {data['nombre'][asimetricoIPost]} \n  ")
if len(data['nombre'][data['simetriaIz'] >50])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Acentuada +50%\n  {data['nombre'][data['simetriaIz'] >50]}\n")

#CELDA DE CARGAS
#relacion flexores
data['relacionFlex']=(data['isqDer']/data['isqIzq'])*100-100
data['relacionFlex']=data['relacionFlex'].round(1)
data['relacionFlex']=data['relacionFlex']* data['celda']
#columnas=['nombre','relacionFlex']
simetricoFlex=(data['relacionFlex'] <=10) & (data['relacionFlex'] >=-10) & data['celda']
asimetricoDFlex=(data['relacionFlex'] <-10) & (data['relacionFlex'] >=-25) & data['celda']
asimetricoIFlex=(data['relacionFlex'] >10) & (data['relacionFlex'] <25) & data['celda']

pdf.add_page()
pdf.set_font("Arial",size=10, style="B")
pdf.set_col(2.5)
pdf.set_y(2)
pdf.multi_cell(h=0.35, align='L', w=0, txt=f"2) FUERZA DE MUSLOS\n 2.1) RELACION FLEXORES ")

pdf.set_col(0)
pdf.set_y(2.5)
pdf.cell(2,2,f'Óptimos en isquios \n',ln=False, align='l')
pdf.set_col(2.5)
pdf.cell(2,2,f'Fortalecer isquios D \n',ln=False, align='l')
pdf.set_col(5)
pdf.cell(2,2,f'Fortalecer isquios I \n',ln=True, align='l')

pdf.set_y(4)
pdf.set_font("Arial",size=8)
pdf.set_col(0)
if len(data['nombre'][simetricoFlex])!=0:
  #pdf.multi_cell(h=0.35, align='L', w=0, txt=f" {data['nombre'][simetricoFlex]}")
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f" {data[['nombre','relacionFlex']][simetricoFlex]}\n")
#if len()!=0:

pdf.set_y(4)
pdf.set_col(2.5)
if len(data['nombre'][asimetricoDFlex])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +10%  \n {data[['nombre','relacionFlex']][asimetricoDFlex]}  \n")

if len(data['nombre'][data['relacionFlex']<-25 *data['celda']])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Grave +25% \n  {data[['nombre','relacionFlex']][data['relacionFlex']<-25 *data['celda']]}\n")

pdf.set_y(4)
pdf.set_col(5)
if len(data['nombre'][asimetricoIFlex])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +10% \n {data[['nombre','relacionFlex']][asimetricoIFlex]} \n  ")

if len(data['nombre'][data['relacionFlex'] >25 *data['celda']])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Grave +25% \n  {data[['nombre','relacionFlex']][data['relacionFlex'] >25 *data['celda']]}\n")

#relacion extensores
data['relacionExt']=(data['cuadDer']/data['cuadIzq'])*100-100
data['relacionExt']=data['relacionExt'].round(1)
data['relacionExt']=data['relacionExt']* data['celda']

simetricoExt=(data['relacionExt'] <=10) & (data['relacionExt'] >=-10) & data['celda']
asimetricoDExt=(data['relacionExt'] <-10) & (data['relacionExt'] >=-25) & data['celda']
asimetricoIExt=(data['relacionExt'] >10) & (data['relacionExt'] <25) & data['celda']

pdf.add_page()
pdf.set_font("Arial",size=10, style="B")
pdf.set_col(2.5)
pdf.set_y(2)
pdf.multi_cell(h=0.35, align='L', w=0, txt=f"2.2) RELACION EXTENSORES")

pdf.set_col(0)
pdf.set_y(2.5)
pdf.cell(2,2,f'Óptimos en cuádriceps \n',ln=False, align='l')
pdf.set_col(2.5)
pdf.cell(2,2,f'Fortalecer cuádriceps D \n',ln=False, align='l')
pdf.set_col(5)
pdf.cell(2,2,f'Fortalecer cuádriceps I \n',ln=True, align='l')

pdf.set_y(4)
pdf.set_font("Arial",size=8)
pdf.set_col(0)
if len(data['nombre'][simetricoExt])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f" {data[['nombre','relacionExt']][simetricoExt]}")
pdf.set_y(4)
pdf.set_col(2.5)
if len(data['nombre'][asimetricoDExt])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +10%  \n {data[['nombre','relacionExt']][asimetricoDExt]} \n ")
if len(data['nombre'][data['relacionExt']<-25 * data['celda']]) !=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Grave +25% \n  {data[['nombre','relacionExt']][data['relacionExt']<-25 * data['celda']]}\n")
pdf.set_y(4)
pdf.set_col(5)
if len(data['nombre'][asimetricoIExt])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +10% \n {data[['nombre','relacionExt']][asimetricoIExt]} \n ")
if len(data['nombre'][data['relacionExt'] >25 *data['celda']])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Grave +25% \n  {data[['nombre','relacionExt']][data['relacionExt'] >25 *data['celda']]}\n")


#relacion interpierna derecha
data['criticoD']=(data['cuadDer']+data['isqDer'])*0.06
data['sCriticoD']=(data['cuadDer']+data['isqDer'])*0.3
data['relacionMd']=(data['cuadDer']+data['isqDer'])*0.7-data['cuadDer']
data['relacionMd']=data['relacionMd'].round(1)
data['relacionMd']=data['relacionMd']* data['celda']

data['simetricoMd']=(data['relacionMd'] <=data['criticoD']) & (data['relacionMd'] >=data['criticoD']*-1) & data['celda']
data['asimetricoDIsq']=(data['relacionMd'] <data['criticoD']*-1) & (data['relacionMd'] >=data['sCriticoD']*-1) & data['celda']
data['asimetricoDCuad']=(data['relacionMd'] >data['criticoD']) & (data['relacionMd'] <data['sCriticoD']) & data['celda']
data['sAsimetricoDIsq']=(data['relacionMd'] <=data['sCriticoD']*-1) * data['celda']
data['sAsimetricoDCuad']=(data['relacionMd'] >= data['sCriticoD']) * data['celda']


pdf.add_page()
pdf.set_font("Arial",size=10, style="B")

pdf.set_col(2.5)
pdf.set_y(2)
pdf.multi_cell(h=0.35, align='L', w=0, txt=f"2.3) RELACION MUSLO DERECHO\n  ")


pdf.set_col(0)
pdf.set_y(2.5)
pdf.cell(2,2,f'Óptimos inter pierna D \n',ln=False, align='l')
pdf.set_col(2.5)
pdf.cell(2,2,f'Fortalecer Isquios D \n',ln=False, align='l')
pdf.set_col(5)
pdf.cell(2,2,f'Fortalecer cuádriceps D \n',ln=True, align='l')

pdf.set_y(4)
pdf.set_font("Arial",size=8)
pdf.set_col(0)
if len(data['nombre'][data['simetricoMd']])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f" {data[['nombre','relacionMd']][data['simetricoMd']]}\n")
#pdf.multi_cell(h=0.35, align='L', w=0, txt=f" {data[['nombre','relacionMd','criticoD','sCriticoD']][data['celda']]}")

pdf.set_y(4)
pdf.set_col(2.5)
if len(data['nombre'][data['asimetricoDIsq']])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +6%  \n {data[['nombre','relacionMd']][data['asimetricoDIsq']]} \n  ")
if len(data['nombre'][data['sAsimetricoDIsq']])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Grave +30% \n {data[['nombre','relacionMd']][data['sAsimetricoDIsq']]}\n")

pdf.set_y(4)
pdf.set_col(5)
if len(data['nombre'][data['asimetricoDCuad']])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +6% \n {data[['nombre','relacionMd']][data['asimetricoDCuad']]} \n ")
if len(data['nombre'][data['sAsimetricoDCuad']])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Grave +30% \n {data[['nombre','relacionMd']][data['sAsimetricoDCuad']]}\n")

#relacion interpierna Izquierda
criticoI=(data['cuadIzq']+data['isqIzq'])*0.06

data['relacionMi']=(data['cuadIzq']+data['isqIzq'])*0.7-data['cuadIzq']
data['relacionMi']=data['relacionMi'].round(1)
data['relacionMi']=data['relacionMi']* data['celda']
#columnas=['nombre','relacionMi']
simetricoMi=(data['relacionMi'] <=criticoI) & (data['relacionMi'] >=criticoI*-1) & data['celda']
asimetricoIIsq=(data['relacionMi'] <criticoI*-1) & (data['relacionMi'] >=criticoI*-5) & data['celda']
asimetricoICuad=(data['relacionMi'] >criticoI) & (data['relacionMi'] <criticoI*5) & data['celda']
sAsimetricoIIsq=(data['relacionMi'] <=criticoI*-5) * data['celda']
sAsimetricoICuad=(data['relacionMi'] >= criticoI*5) * data['celda']

pdf.add_page()
pdf.set_font("Arial",size=10, style="B")
pdf.set_col(2.5)
pdf.set_y(2)
pdf.multi_cell(h=0.35, align='L', w=0, txt=f"2.4) RELACION MUSLO IZQUIERDO")

pdf.set_col(0)
pdf.set_y(2.5)
pdf.cell(2,2,f'Óptimos en inter pierna I \n',ln=False, align='l')
pdf.set_col(2.5)
pdf.cell(2,2,f'Fortalecer Isquios I \n',ln=False, align='l')
pdf.set_col(5)
pdf.cell(2,2,f'Fortalecer cuádriceps I \n',ln=True, align='l')



pdf.set_y(4)
pdf.set_font("Arial",size=8)
pdf.set_col(0)
if len(data['nombre'][simetricoMi])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f" {data[['nombre','relacionMi']][simetricoMi]}\n")
pdf.set_y(4)
pdf.set_col(2.5)
if len(data['nombre'][asimetricoIIsq])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +6%  \n {data[['nombre','relacionMi']][asimetricoIIsq]} \n")
if len(data['nombre'][sAsimetricoIIsq])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Grave +30% \n {data[['nombre','relacionMi']][sAsimetricoIIsq]} \n")
pdf.set_y(4)
pdf.set_col(5)
if len(data['nombre'][asimetricoICuad])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"  \nAsimetría Leve +6% \n {data[['nombre','relacionMi']][asimetricoICuad]} \n  ")
if len(data['nombre'][sAsimetricoICuad])!=0:
  pdf.multi_cell(h=0.35, align='L', w=0, txt=f"\nAsimetría Grave +30% \n {data[['nombre','relacionMi']][sAsimetricoICuad]} \n")


pdfFileName = "Vertiz sep 23 final"+'.pdf'
pdf.output(pdfFileName)
shutil.copy(pdfFileName,"/content/drive/Shareddrives/Ova/Vertiz")